In [1]:
from collections import defaultdict

import pandas as pd
from rdkit import Chem

# retrieve info from new ligprep.sdf
new_ligands = Chem.SDMolSupplier('/opt/home/revoli/eva/Interformer/jane_VS/convert/FDA_ligprep_new.sdf')

data = {}
for mol in new_ligands:
    input_bar = mol.GetProp('input')
    data[input_bar] = [mol.GetProp('_Name'), mol.GetProp('Catalog_NO'), mol.GetProp('State Penalty')]

# convert the old to new data
ligands = Chem.SDMolSupplier('/opt/home/revoli/eva/Interformer/jane_VS/infer/5qay_docked.sdf')
new_data = []
for mol in ligands:
    if mol is None:
        row = None
    elif not mol.HasProp('s_epik_input'):
        row = None
    else:
        row = data[mol.GetProp('s_epik_input')]
    #
    new_data.append(row)
len(new_data)

[09:57:57] Warning: molecule is tagged as 3D, but all Z coords are zero and 2D stereo markers have been found, marking the mol as 2D.
[09:57:57] Warning: molecule is tagged as 3D, but all Z coords are zero and 2D stereo markers have been found, marking the mol as 2D.
[09:57:57] Warning: molecule is tagged as 3D, but all Z coords are zero and 2D stereo markers have been found, marking the mol as 2D.
[09:57:57] Warning: molecule is tagged as 3D, but all Z coords are zero and 2D stereo markers have been found, marking the mol as 2D.
[09:57:58] Warning: molecule is tagged as 3D, but all Z coords are zero and 2D stereo markers have been found, marking the mol as 2D.
[09:58:38] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[09:58:38] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[09:58:38] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol a

343049

In [6]:
import pandas as pd
from rdkit import Chem
from tqdm import tqdm

df = pd.read_csv('/opt/home/revoli/eva/Interformer/result/5qay_docked_infer_ensemble.csv')
df = df.drop(columns=['uff_pose_rank', 'pIC50'])
rank_list = df['pose_rank'].tolist()
len(rank_list)

# read new data
data = defaultdict(list)
# new_data=[Name, Catalog_NO, State Penalty]
for rank in tqdm(rank_list):
    data['epik_penalty'].append(new_data[rank][2])
    data['Catalog_NO'].append(new_data[rank][1])
    data['Name'].append(new_data[rank][0])

df['Name'] = data['Name']
df['Catalog_NO'] = data['Catalog_NO']
df['epik_penalty'] = data['epik_penalty']

df = df.astype({'epik_penalty': float})

df = df.sort_values(by=['pred_pIC50'], ascending=False)
df.to_csv('/tmp/5qay_Interformer_withNO.csv', index=False)
df

100%|██████████| 324920/324920 [00:00<00:00, 2204871.21it/s]


,Target,pose_rank,Molecule ID,pred_pIC50,pred_pose,pred_pIC50_var,Name,Catalog_NO,epik_penalty
20082,5qay,21119,Tezacaftor,7.291016,0.542780,0.211594,Tezacaftor,HY-15448,0.0000
213129,5qay,224809,11-?Keto-?beta-?boswellic acid,7.165039,0.783660,0.161025,11-?Keto-?beta-?boswellic acid,HY-N2056,0.0039
285603,5qay,301293,Bafilomycin A1,7.164062,0.628226,0.135773,Bafilomycin A1,HY-100558,0.0000
73073,5qay,76943,(S)-Crizotinib,7.140625,0.346877,0.152873,(S)-Crizotinib,HY-100549,0.4902
178705,5qay,188392,Astragaloside II,7.129883,0.543785,0.165470,Astragaloside II,HY-N0433,0.0000
...,...,...,...,...,...,...,...,...,...
996,5qay,1045,Choline (bitartrate),-9.206055,0.000002,34.571140,Choline (chloride),HY-B1337,0.0000
100940,5qay,106334,Dimercaprol,-9.289062,0.000003,47.441130,Dimercaprol,HY-B1285,2.4918
999,5qay,1048,Choline (bitartrate),-9.407227,0.000002,34.667770,Choline (chloride),HY-B1337,0.0000
139828,5qay,147344,Trimethylamine N-oxide,-9.418457,0.000003,44.368660,Trimethylamine N-oxide,HY-116084,2.1975


In [7]:

print(df['Catalog_NO'].nunique())

# output best sdf for review
print(len(df[df['epik_penalty'] < 0.5]))
df = df.sort_values('pred_pIC50', ascending=False)
best = df.head(n=100)

sdf = Chem.SDWriter('/tmp/5qay_best.sdf')
for pose_rank in best['pose_rank']:
    sdf.write(ligands[pose_rank])
sdf.close()

7244
136683
